In [1]:
import duckdb

con = duckdb.connect(database=':memory:')

In [2]:
RESULTS_CSV = 'cleanml-intersectional_age@55_ppvdd.csv'

In [3]:
con.execute(f"""
    SELECT error, fairness_impact, accuracy_impact, COUNT(*) as count
    FROM '{RESULTS_CSV}' 
    GROUP BY error, fairness_impact, accuracy_impact
    ORDER BY error, fairness_impact DESC, accuracy_impact DESC
""").df()

,error,fairness_impact,accuracy_impact,count
0,missing_values,negative,negative,3
1,missing_values,insignificant,positive,9
2,missing_values,insignificant,insignificant,6
3,outliers,positive,positive,22
4,outliers,positive,negative,8
5,outliers,positive,insignificant,2
6,outliers,negative,positive,18
7,outliers,negative,insignificant,7
8,outliers,insignificant,positive,55
9,outliers,insignificant,negative,3


In [4]:
counts = con.execute(f"""
    SELECT error, fairness_impact, accuracy_impact, COUNT(*) as count
    FROM '{RESULTS_CSV}' 
    GROUP BY error, fairness_impact, accuracy_impact
    ORDER BY error, fairness_impact DESC, accuracy_impact DESC
""").df()

def single(results, error, fairness_impact, accuracy_impact):
    result_slice = results[(results.error == error) & (results.fairness_impact == fairness_impact) & \
            (results.accuracy_impact == accuracy_impact)]
   
    if len(result_slice) > 0:
        return list(result_slice['count'])[0]                        
    else:
        return 0

def perc(count, total):
    if total == 0: return '0.0\% (0)'  # TODO: Check
    return str(round((count / total) * 100, 1)) + f'\% ({count})'

In [5]:
for error in ['missing_values', 'outliers', 'mislabel']:
    print('%', error)
    cpn = single(counts, error, 'positive', 'negative')
    cpi = single(counts, error, 'positive', 'insignificant')
    cpp = single(counts, error, 'positive', 'positive')

    cin = single(counts, error, 'insignificant', 'negative')
    cii = single(counts, error, 'insignificant', 'insignificant')
    cip = single(counts, error, 'insignificant', 'positive')            

    cnn = single(counts, error, 'negative', 'negative')
    cni = single(counts, error, 'negative', 'insignificant')
    cnp = single(counts, error, 'negative', 'positive')

    total = cpn + cpi + cpp + cin + cii + cip + cnn + cni + cnp        

    print('& negative & insign. & positive & \\\\')    
    print('\\hline')
    print('worse &', perc(cnn, total), '&', perc(cni, total), '&', perc(cnp, total), '&', perc(cnn + cni + cnp, total),'\\\\')
    print('insign. &', perc(cin, total), '&', perc(cii, total), '&', perc(cip, total), '&', perc(cin + cii + cip, total),'\\\\')
    print('better &', perc(cpn, total), '&', perc(cpi, total), '&', perc(cpp, total), '&', perc(cpn + cpi + cpp, total),'\\\\')
    print('\\hline')
    print(' &', perc(cpn + cin + cnn, total), '&', perc(cpi + cii + cni, total), '&', perc(cpp+cip+cnp, total), '& \\\\')

% missing_values
& negative & insign. & positive & \\
\hline
worse & 16.7\% (3) & 0.0\% (0) & 0.0\% (0) & 16.7\% (3) \\
insign. & 0.0\% (0) & 33.3\% (6) & 50.0\% (9) & 83.3\% (15) \\
better & 0.0\% (0) & 0.0\% (0) & 0.0\% (0) & 0.0\% (0) \\
\hline
 & 16.7\% (3) & 33.3\% (6) & 50.0\% (9) & \\
% outliers
& negative & insign. & positive & \\
\hline
worse & 0.0\% (0) & 4.3\% (7) & 11.1\% (18) & 15.4\% (25) \\
insign. & 1.9\% (3) & 29.0\% (47) & 34.0\% (55) & 64.8\% (105) \\
better & 4.9\% (8) & 1.2\% (2) & 13.6\% (22) & 19.8\% (32) \\
\hline
 & 6.8\% (11) & 34.6\% (56) & 58.6\% (95) & \\
% mislabel
& negative & insign. & positive & \\
\hline
worse & 0.0\% (0) & 0.0\% (0) & 0.0\% (0) & 0.0\% (0) \\
insign. & 0.0\% (0) & 0.0\% (0) & 0.0\% (0) & 0.0\% (0) \\
better & 0.0\% (0) & 0.0\% (0) & 0.0\% (0) & 0.0\% (0) \\
\hline
 & 0.0\% (0) & 0.0\% (0) & 0.0\% (0) & \\


## How does this differ by dataset?

In [6]:
con.execute(f"""
    SELECT 
        dataset,
        error,    
        fairness_impact!='negative' as non_negative_impact_on_fairness, 
        COUNT(*) as count    
    FROM '{RESULTS_CSV}' 
    GROUP BY dataset, error, non_negative_impact_on_fairness
    ORDER BY dataset, error, non_negative_impact_on_fairness DESC
""").df()

,dataset,error,non_negative_impact_on_fairness,count
0,adult,missing_values,True,6
1,adult,outliers,True,17
2,adult,outliers,False,10
3,folktables,missing_values,True,3
4,folktables,missing_values,False,3
5,folktables,outliers,True,21
6,folktables,outliers,False,6
7,german,missing_values,True,6
8,german,outliers,True,75
9,german,outliers,False,6


## For which scenarios (dataset, error and metric) did we find a cleaning approach with non-neg impact on  fairness?

In [7]:
con.execute(f"""
    SELECT DISTINCT dataset, error, criteria, metric
    FROM '{RESULTS_CSV}' as csv
    WHERE EXISTS(
        SELECT dataset, error, criteria, metric FROM '{RESULTS_CSV}' AS csv2
        WHERE 
            csv2.criteria = csv.criteria AND
            csv2.dataset = csv.dataset AND
            csv2.error = csv.error AND
            csv2.metric = csv.metric AND
            csv2.fairness_impact != 'negative'
    )    
    ORDER BY dataset, error, criteria, metric
""").df()

,dataset,error,criteria,metric
0,adult,missing_values,sex/race,equal_opportunity
1,adult,outliers,sex/race,equal_opportunity
2,folktables,missing_values,sex/rac1p,equal_opportunity
3,folktables,outliers,sex/rac1p,equal_opportunity
4,german,missing_values,age/sex,equal_opportunity
5,german,outliers,age/foreign_worker,equal_opportunity
6,german,outliers,age/sex,equal_opportunity
7,german,outliers,sex/foreign_worker,equal_opportunity
8,heart,outliers,gender/age,equal_opportunity


## For which scenarios (dataset, error and metric) did we find a cleaning approach with positive impact on  fairness?

In [8]:
con.execute(f"""
    SELECT DISTINCT dataset, metric, criteria, error
    FROM '{RESULTS_CSV}' as csv
    WHERE EXISTS(
        SELECT dataset, error, criteria, metric FROM '{RESULTS_CSV}' AS csv2
        WHERE 
            csv2.criteria = csv.criteria AND
            csv2.dataset = csv.dataset AND
            csv2.error = csv.error AND
            csv2.metric = csv.metric AND
            csv2.fairness_impact == 'positive'
    )    
    ORDER BY dataset, metric, error, criteria
""").df()

,dataset,metric,criteria,error
0,adult,equal_opportunity,sex/race,outliers
1,german,equal_opportunity,age/foreign_worker,outliers
2,german,equal_opportunity,age/sex,outliers
3,heart,equal_opportunity,gender/age,outliers


## For which scenarios (dataset, error and metric) did we find a cleaning approach to improve fairness and accuracy?

In [9]:
con.execute(f"""
    SELECT DISTINCT dataset, metric, criteria, error
    FROM '{RESULTS_CSV}' as csv
    WHERE EXISTS(
        SELECT dataset, error, criteria, metric FROM '{RESULTS_CSV}' AS csv2
        WHERE 
            csv2.criteria = csv.criteria AND    
            csv2.dataset = csv.dataset AND
            csv2.error = csv.error AND
            csv2.metric = csv.metric AND
            csv2.fairness_impact = 'positive' AND csv2.accuracy_impact='positive'
    )   
    ORDER BY dataset, metric, error, criteria
""").df()

,dataset,metric,criteria,error
0,adult,equal_opportunity,sex/race,outliers
1,german,equal_opportunity,age/foreign_worker,outliers
2,german,equal_opportunity,age/sex,outliers
3,heart,equal_opportunity,gender/age,outliers


In [10]:
con.execute(f"""
    SELECT DISTINCT model, COUNT(*) AS count
    FROM '{RESULTS_CSV}' 
    WHERE fairness_impact == 'negative'
    GROUP BY model
    ORDER BY model
""").df()

,model,count
0,XGBoost,5
1,knn_classification,7
2,logistic_regression,16


In [11]:
con.execute(f"""
    SELECT DISTINCT model, COUNT(*) AS count
    FROM '{RESULTS_CSV}' 
    WHERE fairness_impact != 'negative'
    GROUP BY model
    ORDER BY model
""").df()

,model,count
0,XGBoost,49
1,knn_classification,47
2,logistic_regression,56


In [12]:
con.execute(f"""
    SELECT DISTINCT model, COUNT(*) AS count
    FROM '{RESULTS_CSV}' 
    WHERE fairness_impact = 'positive'
    GROUP BY model
    ORDER BY model
""").df()

,model,count
0,XGBoost,16
1,knn_classification,3
2,logistic_regression,13


In [13]:
con.execute(f"""
    SELECT DISTINCT model, COUNT(*) AS count
    FROM '{RESULTS_CSV}' 
    WHERE fairness_impact = 'positive' AND accuracy_impact = 'positive'
    GROUP BY model
    ORDER BY model
""").df()

,model,count
0,XGBoost,7
1,knn_classification,3
2,logistic_regression,12


In [14]:
con.execute(f"""
    SELECT DISTINCT model, COUNT(*) AS count
    FROM '{RESULTS_CSV}' 
    WHERE fairness_impact = 'positive' AND accuracy_impact = 'positive'
    GROUP BY model
    ORDER BY model
""").df()

,model,count
0,XGBoost,7
1,knn_classification,3
2,logistic_regression,12


In [15]:
con.execute(f"""
    SELECT metric, fairness_impact, COUNT(*) AS count, COUNT(*) / 318.0 as perc
    FROM '{RESULTS_CSV}' 
    GROUP BY metric, fairness_impact
    ORDER BY metric, fairness_impact
""").df()

,metric,fairness_impact,count,perc
0,equal_opportunity,insignificant,120,0.377358
1,equal_opportunity,negative,28,0.088050
2,equal_opportunity,positive,32,0.100629


In [16]:
con.execute(f"""
    SELECT dataset, metric, count(*) as count
    FROM '{RESULTS_CSV}' 
    WHERE error = 'missing_values' AND fairness_impact='negative' AND accuracy_impact='insignificant'
    GROUP BY dataset, metric
    ORDER BY dataset, metric
""").df()

,dataset,metric,count


In [17]:
con.execute(f"""
    SELECT dataset, metric, count(*) as count
    FROM '{RESULTS_CSV}' 
    WHERE error = 'missing_values' AND fairness_impact='positive' AND accuracy_impact='positive'
    GROUP BY dataset, metric
    ORDER BY dataset, metric
""").df()

,dataset,metric,count
